In [11]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image


In [12]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Définir le chemin vers le dataset
dataset_path = "./D3_Final"

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,  # Increased rotation
    width_shift_range=0.3,  # More shifts
    height_shift_range=0.3,
    shear_range=0.2,  # Added shear
    zoom_range=0.3,  # Increased zoom
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],  # Brightness adjustment
    fill_mode='nearest'
)


# Lister toutes les images avec leur classe
image_paths = []
labels = []
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):  # Vérifier que c'est un dossier
        for img in os.listdir(class_path):
            image_paths.append(os.path.join(class_name, img))  # Chemin relatif
            labels.append(class_name)

# Créer un DataFrame pandas
df = pd.DataFrame({"filename": image_paths, "class": labels})

# Diviser en train (80%), validation (10%) et test (10%)
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df["class"], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["class"], random_state=42)

# Définir ImageDataGenerator pour chaque dataset
datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                             height_shift_range=0.2, horizontal_flip=True, zoom_range=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)  # Pas d'augmentation pour le test

# Charger les datasets avec flow_from_dataframe
train_data = datagen.flow_from_dataframe(train_df, directory=dataset_path,
                                         x_col="filename", y_col="class",
                                         target_size=(256, 256), batch_size=32, class_mode="categorical")

val_data = test_datagen.flow_from_dataframe(val_df, directory=dataset_path,
                                            x_col="filename", y_col="class",
                                            target_size=(256, 256), batch_size=32, class_mode="categorical")

test_data = test_datagen.flow_from_dataframe(test_df, directory=dataset_path,
                                             x_col="filename", y_col="class",
                                             target_size=(256, 256), batch_size=32, class_mode="categorical")

# Get the number of classes (wheat disease types)
num_classes = len(train_data.class_indices)




Found 1670 validated image filenames belonging to 3 classes.
Found 211 validated image filenames belonging to 3 classes.
Found 210 validated image filenames belonging to 3 classes.


c:\Users\DELL\Documents\PFE\M3\venv\Lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 58 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
c:\Users\DELL\Documents\PFE\M3\venv\Lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 5 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
c:\Users\DELL\Documents\PFE\M3\venv\Lib\site-packages\keras\src\legacy\preprocessing\image.py:920: UserWarning: Found 7 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [13]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import (
    Input, Conv2D, SeparableConv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
)
from tensorflow.keras.models import Model

# Number of classes (based on the diagram: Yellow Rust, Brown Rust, Mildew, Septoria, Healthy)
num_classes = 3

# Load EfficientNetB0 as feature extractor
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(256, 256, 3))
base_model.trainable = True  # Allow fine-tuning

# Freeze initial layers (to match the training strategy)
for layer in base_model.layers[:150]:  
    layer.trainable = False

# Define CropNet Model
inputs = Input(shape=(256, 256, 3))
x = base_model(inputs, training=False)

# Custom CNN layers for feature refinement (matching the diagram)
x = SeparableConv2D(32, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = SeparableConv2D(64, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = SeparableConv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

# Fully connected layers (as shown in the diagram)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

# Output layer for classification
outputs = Dense(num_classes, activation='softmax')(x)

# Create model
model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])

# Show model summary
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 8, 8, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d                │ (None, 8, 8, 32)       │        52,512 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 8, 8, 32)       │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 4, 4, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_1              │ (None, 4, 4, 64)       │         2,400 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 4, 4, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 2, 2, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_2              │ (None, 2, 2, 128)      │         8,896 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 2, 2, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,182,118 (15.95 MB)

 Trainable params: 3,419,035 (13.04 MB)

 Non-trainable params: 763,083 (2.91 MB)

In [16]:
# ================================
# 3️⃣ Train the Model
# ================================

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)
    ]
)

Epoch 1/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 98s 2s/step - accuracy: 0.5611 - loss: 1.0250 - val_accuracy: 0.5498 - val_loss: 0.9835 - learning_rate: 5.0000e-04
Epoch 2/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 101s 2s/step - accuracy: 0.5788 - loss: 1.0146 - val_accuracy: 0.4408 - val_loss: 1.2057 - learning_rate: 5.0000e-04
Epoch 3/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 123s 2s/step - accuracy: 0.5861 - loss: 0.9471 - val_accuracy: 0.6540 - val_loss: 0.8406 - learning_rate: 5.0000e-04
Epoch 4/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - accuracy: 0.5808 - loss: 0.9649 - val_accuracy: 0.4739 - val_loss: 1.0238 - learning_rate: 5.0000e-04
Epoch 5/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 127s 2s/step - accuracy: 0.5932 - loss: 0.9833 - val_accuracy: 0.4787 - val_loss: 1.0056 - learning_rate: 5.0000e-04
Epoch 6/100
53/53 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - accuracy: 0.6043 - loss: 0.9121 - val_accuracy: 0.4455 - val_loss: 1.1461 - learning_rate: 2.5000e-04


In [17]:
# ================================
# 4️⃣ Evaluate the Model on Test Data
# ================================

test_loss, test_acc = model.evaluate(test_data)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")


7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 373ms/step - accuracy: 0.6617 - loss: 0.8213

✅ Test Accuracy: 0.6667
